# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a video.




## Install OpenPose

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`



Selecting previously unselected package liblmdb0:amd64.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../00-liblmdb0_0.9.24-1build2_amd64.deb ...
Unpacking liblmdb0:amd64 (0.9.24-1build2) ...
Selecting previously unselected package libgflags2.2.
Preparing to unpack .../01-libgflags2.2_2.2.2-2_amd64.deb ...
Unpacking libgflags2.2 (2.2.2-2) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../02-libgflags-dev_2.2.2-2_amd64.deb ...
Unpacking libgflags-dev (2.2.2-2) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../03-libgoogle-glog0v5_0.5.0+really0.4.0-2_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.5.0+really0.4.0-2) ...
Selecting previously unselected package libunwind-dev:amd64.
Preparing to unpack .../04-libunwind-dev_1.3.2-2build2.1_amd64.deb ...
Unpacking libunwind-dev:amd64 (1.3.2-2build2.1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to un

In [ ]:
##run OpenPose on videos

!rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video /content/T7.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi outputT7.mp4  ##Change for specifc video

In [ ]:
import time

# Number of iterations
n = 109

# Empty list to store execution times
times = []

for i in range(n):
    start_time = time.time()  # Get start time
    # Code to be timed
    !rm openpose.avi
    T_dir = '/content/T'+str(i+1)+'.mp4'
    !cd openpose && ./build/examples/openpose/openpose.bin --video $T_dir --write_json ./output/ --display 0  --write_video ../openpose.avi
    output_mp4 = 'outputT'+str(i+1)+'.mp4'
    !ffmpeg -y -loglevel info -i openpose.avi $output_mp4  ##Change for specifc video
    end_time = time.time()  # Get end time
    execution_time = end_time - start_time  # Calculate execution time
    times.append(execution_time)  # Add execution time to list

# Calculate statistics
mean_time = sum(times) / n
max_time = max(times)
min_time = min(times)

# Print statistics
print(f"Mean time: {mean_time:.6f} seconds")
print(f"Maximum time: {max_time:.6f} seconds")
print(f"Minimum time: {min_time:.6f} seconds")

# Run this code cell to make directory 'content/openpose/output/' where you will have to upload all the files from the Google drive link to under the name 'openpose_output'.

In [ ]:
# !rm -rf /content/openpose

In [ ]:
!mkdir /content/openpose/
# !mkdir /content/openpose/output/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/tackle_videos/openpose_output/output /content/openpose/output

#Assigning unique IDs to each player

In [ ]:
import cv2
import json
import math
import random
import glob, os
import cv2
import re


!mkdir /content/output/
!mkdir /content/output_with_ids/

def assignIDs(video,starts_with_number):

  !rm -r /content/images
  !mkdir /content/images
  dir = '/content/output_with_ids/'+video
  !mkdir $dir

  path = '/content/'+video ##CHANGE
  cap = cv2.VideoCapture(path)
  length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  print(length)
  fps = cap.get(cv2.CAP_PROP_FPS)
  frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
  new_video_name = 'neck_positions'+video
  out = cv2.VideoWriter(new_video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, frame_size)


  for inde in range(length):
    cap.set(cv2.CAP_PROP_POS_FRAMES, int(inde)-1)
    res, frame = cap.read()
    image_path_const = '/content/images/test'+str(inde)+'.png'
    cv2.imwrite(image_path_const, frame)

  # set the path to your folder
  folder_path = "/content/openpose/output/"
  starts_with = video[:starts_with_number]+'_'

  # iterate through each file in the folder
  for filename in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filename)
      if os.path.isfile(file_path) and filename.startswith(starts_with):
          # print the filename if it starts with 'T1'
          file_loc = '/content/openpose/output/'+filename
          !cp -R $file_loc $dir

  vcap = cv2.VideoCapture(video) # 0=camera changeeee

  if vcap.isOpened():
      # get vcap property
      width  = vcap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
      height = vcap.get(cv2.CAP_PROP_FRAME_HEIGHT)

  # Initialize a dictionary to keep track of the used IDs and their last known positions
  used_ids = {}
  id_list = []

  def assign_id(player):
      """
      Assigns a unique ID to the given player, taking into account the previously used IDs and their positions.
      """
      x, y = player['pose_keypoints_2d'][3], player['pose_keypoints_2d'][4]

      # Initialize the closest ID to None and the minimum distance to infinity
      closest_id = None
      min_distance = math.inf

      # Iterate over the used IDs and their last known positions
      for id_, position in used_ids.items():
          # Calculate the distance between the current player and the last known position of the ID
          distance = math.sqrt((x - position[0])**2 + (y - position[1])**2)
          if distance < min_distance:
              # If the distance is smaller than the minimum distance, update the closest ID and minimum distance
              closest_id = id_
              min_distance = distance

      # If a closest ID was found and the minimum distance is less than some threshold (e.g., 50 pixels), assign it to the player
      if closest_id is not None and min_distance < 50:
          player['person_id'] = closest_id
          used_ids[closest_id] = [x,y]  # Update the last known position of the ID
      else:
          # If no closest ID was found or the minimum distance is too large, generate a new ID

          new_id = get_new_id(id_list)
          player['person_id'] = new_id
          used_ids[new_id] = [x,y]  # Add the new ID and its position to the used IDs
          id_list.append(new_id)

      return player

  def get_new_id(id_list):
      if not id_list:
          return 1
      else:
          max_id = max(id_list)
          return max_id + 1

  numbers = re.compile(r'(\d+)')
  def numericalSort(value):
      parts = numbers.split(value)
      parts[1::2] = map(int, parts[1::2])
      return parts

  path_image = '/content/images'
  image_list = []
  for filename in sorted(glob.glob(os.path.join(path_image, '*.png')), key=numericalSort):

    image_list.append(filename)
  # Loop through input files
  path_poses = dir
  ic = 0
  image_number = 0

  for filename in sorted(glob.glob(os.path.join(path_poses, '*.json')), key=numericalSort):

    new_keypoints_dict = {}
    with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
        distros_dict = json.load(f)

    img = cv2.imread(image_list[ic])
    all_keypoints = []
    for distro in distros_dict['people']:

      all_keypoints.append(distro['pose_keypoints_2d'])
    ids = []

    for idx, p in enumerate(distros_dict['people']):
      ids.append(idx)

      if p['person_id'] == None:
        continue

      new_keypoints = assign_id(p)

      new_keypoints_dict[new_keypoints['person_id']] = new_keypoints['pose_keypoints_2d']
    # Assign person IDs based on proximity to previous poses

    poses = all_keypoints

    # Draw pose keypoints and person IDs on image

    img = cv2.imread(image_list[image_number])

    for id,pose in new_keypoints_dict.items():

        x, y = int(pose[3]), int(pose[4])
        cv2.putText(img,str(id), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.circle(img, (x, y), 5, (0, 0, 255), -1)

    # Show image
    from google.colab.patches import cv2_imshow
    # cv2_imshow(img)
    out.write(img)  # Add frame to video writer

    cv2.waitKey(0)

    image_number += 1

    # Update distros_dict with new person IDs
    for person in distros_dict['people']:
      person_id = person['person_id']
      if person_id in new_keypoints_dict:
        person['person_id'] = person_id
    # Save updated distros_dict back to the same JSON file
    with open(os.path.join(os.getcwd(), filename), 'w') as f:
        json.dump(distros_dict, f)
  # Release resources
  out.release()
  cap.release()

#Do assignments in batches so that you can change starts_with variable when numbers are over 1 or 2 digits.

In [ ]:
import cv2
import json
import math
import random
import glob, os
import cv2
import re

def process_all_files():
    starts_with_number = 2
    for i in range(1, 10):
        filename = 'T' + str(i) + '.mp4'
        print(filename)
        assignIDs(filename, starts_with_number)
process_all_files()

T1.mp4
rm: cannot remove '/content/images': No such file or directory
15
T2.mp4
40
T3.mp4
55
T4.mp4
40
T5.mp4
27
T6.mp4
14
T7.mp4
22
T8.mp4
25
T9.mp4
27


In [ ]:
def process_all_files():
    starts_with_number = 3
    for i in range(10, 51):
        filename = 'T' + str(i) + '.mp4'
        print(filename)
        assignIDs(filename, starts_with_number)
process_all_files()

In [ ]:
def process_all_files():
    starts_with_number = 3
    for i in range(51, 100):
        filename = 'T' + str(i) + '.mp4'
        print(filename)
        assignIDs(filename, starts_with_number)
process_all_files()

In [ ]:
def process_all_files():
    starts_with_number = 4
    for i in range(100, 110):
        filename = 'T' + str(i) + '.mp4'
        print(filename)
        assignIDs(filename, starts_with_number)
process_all_files()

#Identify who the ball carrier is in the first frame of each video


In [ ]:

# Import necessary libraries
import os, glob
import re
import json
from scipy import spatial

# Define a function to get the index of the ball carrier based on their neck keypoints and ball position
def get_ball_carrier_id(neck_keypoints, ball_x, ball_y):
  # Create a KDTree to efficiently search for the nearest point
  tree = spatial.KDTree(neck_keypoints)

  # Convert ball position to integers
  X = int(ball_x)
  Y = int(ball_y)

  # Query the KDTree for the nearest point to the ball position
  high = tree.query([(X,Y)])

  # Get the index of the nearest point
  ind = high[1][0]

  return ind

# Define a function to sort filenames numerically
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

# Define a function to get all neck and hip keypoints for the ball carrier in all frames
def getALLNeckAndHipKeypointsBallCarrier(path, x_ball, y_ball):
  count = 0
  x_neck_keypoints = []
  y_neck_keypoints = []
  x_hip_keypoints = []
  y_hip_keypoints = []
  ball_carrier_index_list = []
  ball_carrier_id = 0

  # Loop through all JSON files in the specified path
  for filename in sorted(glob.glob(os.path.join(path, '*.json')), key=numericalSort):
    with open(os.path.join(os.getcwd(), filename), 'r') as f: # Open the file in read-only mode
      distros_dict = json.load(f) # Load the JSON data into a dictionary

    all_x_neck_keypoints = []
    all_y_neck_keypoints = []

    true = 0

    # Loop through all people detected in the frame
    for distro in distros_dict['people']:
      all_x_neck_keypoints.append(distro['pose_keypoints_2d'][3]) # Append the x-coordinate of the neck keypoint to the list
      all_y_neck_keypoints.append(distro['pose_keypoints_2d'][4]) # Append the y-coordinate of the neck keypoint to the list

    all_neck_keypoints = list(zip(all_x_neck_keypoints, all_y_neck_keypoints)) # Combine x and y coordinates into a list of tuples

    if count == 0: # If this is the first frame, identify the ball carrier
      ball_Carrier_index = get_ball_carrier_id(all_neck_keypoints, x_ball, y_ball) # Get the index of the ball carrier
      ball_carrier_id = distros_dict['people'][ball_Carrier_index]['person_id'] # Get the ID of the ball carrier

    person_check = []
    # Loop through all people detected in the frame again
    for distro in distros_dict['people']:
      if distro['person_id'] == ball_carrier_id: # If this person is the ball carrier
        # Append their neck and hip keypoints to the respective lists
        x_neck_keypoints.append(distro['pose_keypoints_2d'][3])
        y_neck_keypoints.append(distro['pose_keypoints_2d'][4])
        x_hip_keypoints.append(distro['pose_keypoints_2d'][24])
        y_hip_keypoints.append(distro['pose_keypoints_2d'][25])

        person_check.append(distro['person_id'])
      else:
        person_check.append(distro['person_id'])

    if ball_carrier_id not in person_check:

      x_neck_keypoints.append(None)
      y_neck_keypoints.append(None)
      x_hip_keypoints.append(None)
      y_hip_keypoints.append(None)


    ball_carrier_index_list.append(ball_carrier_id)
    count += 1

  neck_keypoints_ball_carrier = list(zip(x_neck_keypoints, y_neck_keypoints))
  hip_keypoints_ball_carrier = list(zip(x_hip_keypoints, y_hip_keypoints))

  return [neck_keypoints_ball_carrier, hip_keypoints_ball_carrier, ball_carrier_index_list]

#Determine the direction of the ball carrier

In [ ]:
# Import math Library
import math

# This function calculates the speed of the ball carrier and returns a list of directions.
def get_speed(keypoints, hipKeypoints):
  first = 0
  direction = 0
  direction_avg = []
  prev_direction = 0

  # Loop over all keypoints and hipKeypoints.
  for point in range(len(keypoints)):
    # If the keypoint or hipKeypoint is None, skip it.
    if keypoints[point][0]==None or hipKeypoints[point][0]==None:
      continue

    # If the keypoint is on the right side of the hipKeypoint, set the direction to 1 (RIGHT).
    if keypoints[point][0] > hipKeypoints[point][0]:
      direction = 1 ##RIGHT

    # If the keypoint is on the same horizontal line as the hipKeypoint, set the direction to the previous direction.
    elif keypoints[point][0] == hipKeypoints[point][0]:
      direction = prev_direction ##CASE HANDLING

    # If the keypoint is on the left side of the hipKeypoint, set the direction to 0 (LEFT).
    else:
      direction = 0 ##LEFT

    # Set the prev_direction to the current direction.
    prev_direction = direction
    # Append the direction to the direction_avg list.
    direction_avg.append(direction)

  # Return the direction_avg list.
  return direction_avg


#Save neck and hip keypoints of all players in the video into a dictionary

In [ ]:
#####GET ALL PERSONS POSES#####


# Import json, os and glob libraries
import json
import os, glob
import re

# Define a regular expression to match numbers in a string
numbers = re.compile(r'(\d+)')

# Define a function to sort filenames numerically
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

# Define a function to get all persons' poses
def getAllPersonsPoses():
  count=0
  x_neck_keypoints_dict = {}
  y_neck_keypoints_dict = {}
  x_hip_keypoints_dict = {}
  y_hip_keypoints_dict = {}
  distinct_people = []

  # Loop through all the JSON files in the given path, sorted numerically
  for filename in sorted(glob.glob(os.path.join(path, '*.json')), key=numericalSort):
    with open(os.path.join(os.getcwd(), filename), 'r') as f: # Open the file in readonly mode
      distros_dict = json.load(f)  # Load the JSON data from the file

    true = 0
    x_neck_list = []
    x_neck_keypoints_dict[count] = {}
    y_neck_keypoints_dict[count] = {}
    x_hip_keypoints_dict[count] = {}
    y_hip_keypoints_dict[count] = {}
    distinct_people_check = []

    # Loop through each person in the JSON data
    for distro in distros_dict['people']:
      # If the person is not already recorded, record their pose keypoints
      if distinct_people.count(distro['person_id']) == 0:
        distinct_people.append(distro['person_id'])

        x_neck_keypoints_dict[count][distro['person_id']] = distro['pose_keypoints_2d'][3]
        y_neck_keypoints_dict[count][distro['person_id']] = distro['pose_keypoints_2d'][4]
        x_hip_keypoints_dict[count][distro['person_id']] = distro['pose_keypoints_2d'][24]
        y_hip_keypoints_dict[count][distro['person_id']] = distro['pose_keypoints_2d'][25]
        distinct_people_check.append(distro['person_id'])

      # If the person is already recorded, update their pose keypoints
      else:
        x_neck_keypoints_dict[count][distro['person_id']] = distro['pose_keypoints_2d'][3]
        y_neck_keypoints_dict[count][distro['person_id']] = distro['pose_keypoints_2d'][4]
        x_hip_keypoints_dict[count][distro['person_id']] = distro['pose_keypoints_2d'][24]
        y_hip_keypoints_dict[count][distro['person_id']] = distro['pose_keypoints_2d'][25]
        distinct_people_check.append(distro['person_id'])

    # If there are still persons who haven't been recorded, record their pose keypoints as None
    if len(list(set(distinct_people)-set(distinct_people_check)))>0:
      additional_list = list(set(distinct_people)-set(distinct_people_check))
      for i in additional_list:
        x_neck_keypoints_dict[count][i] = None
        y_neck_keypoints_dict[count][i] = None
        x_hip_keypoints_dict[count][i] = None
        y_hip_keypoints_dict[count][i] = None

    count += 1

  for key in x_neck_keypoints_dict:
    check_list = []
    for value in x_neck_keypoints_dict[key]:
      check_list.append(value)
    if len(list(set(distinct_people)-set(check_list)))>0:
      additional_list = list(set(distinct_people)-set(check_list))
      for i in additional_list:
        x_neck_keypoints_dict[key][i] = None
        y_neck_keypoints_dict[key][i] = None

  for key in x_hip_keypoints_dict:
    check_list = []
    for value in x_hip_keypoints_dict[key]:
      check_list.append(value)
    if len(list(set(distinct_people)-set(check_list)))>0:
      additional_list = list(set(distinct_people)-set(check_list))
      for i in additional_list:
        x_hip_keypoints_dict[key][i] = None
        y_hip_keypoints_dict[key][i] = None
  return [distinct_people, x_neck_keypoints_dict, y_neck_keypoints_dict, x_hip_keypoints_dict, y_hip_keypoints_dict]



#Combine the x and y coordinates of the keypoints together into a dictionary to be later used to assign them to IDs

In [ ]:
#####GET X AND Y COORDINATES OF EACH FRAME#####
def combineXYdict(x_neck_keypoints_dict, y_neck_keypoints_dict, x_hip_keypoints_dict, y_hip_keypoints_dict):
  neck_points_dict = {}

  for fram, x_neck in x_neck_keypoints_dict.items():
    x_neck_points = []
    y_neck_points = []
    for x in range(len(x_neck)):

      x_neck_points.append(x_neck_keypoints_dict[fram][x+1])
      y_neck_points.append(y_neck_keypoints_dict[fram][x+1])
    neck_points3 = list(zip(x_neck_points, y_neck_points))
    neck_points_dict['frame '+str(fram+1)] = neck_points3

  hip_points_dict = {}

  for fram, x_hip in x_hip_keypoints_dict.items():
    x_hip_points = []
    y_hip_points = []
    for x in range(len(x_hip)):
      x_hip_points.append(x_hip_keypoints_dict[fram][x+1])
      y_hip_points.append(y_hip_keypoints_dict[fram][x+1])
    hip_points3 = list(zip(x_hip_points, y_hip_points))
    hip_points_dict['frame '+str(fram+1)] = hip_points3

  return [neck_points_dict, hip_points_dict]


#Assign coordinates to IDs

In [ ]:

#####GET X AND Y COORDINATES OF EACH POSE TOGETHER#####
def separateKeypointsEachId(distinct_people, x_neck_keypoints_dict, y_neck_keypoints_dict, x_hip_keypoints_dict, y_hip_keypoints_dict):
  neck_keypoints3_dict = {}
  hip_keypoints3_dict = {}


  for n in distinct_people:
    x_neck_keypoints3 = []
    y_neck_keypoints3 = []
    x_hip_keypoints3 = []
    y_hip_keypoints3 = []
    for id in x_neck_keypoints_dict:
      x_neck_keypoints3.append(x_neck_keypoints_dict[id][n])
      y_neck_keypoints3.append(y_neck_keypoints_dict[id][n])

    for id2 in x_hip_keypoints_dict:
      x_hip_keypoints3.append(x_hip_keypoints_dict[id2][n])
      y_hip_keypoints3.append(y_hip_keypoints_dict[id2][n])

    neck_keypoints3 = list(zip(x_neck_keypoints3, y_neck_keypoints3))
    neck_keypoints3_dict[n] = neck_keypoints3

    hip_keypoints3 = list(zip(x_hip_keypoints3, y_hip_keypoints3))
    hip_keypoints3_dict[n] = hip_keypoints3

  return [neck_keypoints3_dict, hip_keypoints3_dict]


#Check in every frame if any players are within the circle of the ball-carrier created around their neck

In [ ]:

def calculateNewOverlap(neck_points_dict, neck_keypoints3_dict, ball_carrier_index_list_global, ball_carrier_direction_overall, neck_keypoints_ball_carrier, hip_keypoints3_dict):
  opposite_direction_players_overlapping2 = [] # Create an empty list to store overlapping players
  radius = int(150) # Set the radius of the circle used for overlap calculation

  # Loop through each frame in the input data
  for i in range(len(neck_points_dict)):

    # Loop through each player's neck keypoints in the current frame
    for point in neck_keypoints3_dict:

      # Skip the ball carrier and any keypoints with missing values
      if point != ball_carrier_index_list_global[i]:

        if neck_keypoints3_dict[point][i][0] == None:
          continue

        # Check if the ball carrier is moving left to right
        if ball_carrier_direction_overall == 1:

          # Skip if ball carrier's keypoint or current keypoint is missing, or if current keypoint is too far to the left
          if neck_keypoints_ball_carrier[i][0] == None:
            continue
          if neck_keypoints3_dict[point][i][0] + 70 < neck_keypoints_ball_carrier[i][0]:
            continue

          # Calculate distance between the ball carrier and current keypoint, and check if it's within the radius
          if (neck_keypoints3_dict[point][i][0] - neck_keypoints_ball_carrier[i][0])**2 + (neck_keypoints3_dict[point][i][1] - neck_keypoints_ball_carrier[i][1])**2 <= radius**2:

            # Check if there is no data available for the next frame for either the current keypoint or the ball carrier
            if i + 1 >= len(neck_points_dict):
              opposite_direction_players_overlapping2.append(['frame '+str(i), point])
              break
            if neck_keypoints3_dict[point][i+1][0] == None:
              opposite_direction_players_overlapping2.append(['frame '+str(i), point])
            if neck_keypoints_ball_carrier[i+1][0] == None:
              opposite_direction_players_overlapping2.append(['frame '+str(i), point])

        # Check if the ball carrier is moving right to left
        else:
          # Skip if ball carrier's keypoint or current keypoint is missing, or if current keypoint is too far to the right
          if neck_keypoints_ball_carrier[i][0] == None:
            continue
          if neck_keypoints3_dict[point][i][0] - 70 > neck_keypoints_ball_carrier[i][0]:
            continue

          # Calculate distance between the ball carrier and current keypoint, and check if it's within the radius
          if (neck_keypoints3_dict[point][i][0] - neck_keypoints_ball_carrier[i][0])**2 + (neck_keypoints3_dict[point][i][1] - neck_keypoints_ball_carrier[i][1])**2 <= radius**2:

            # Check if there is no data available for the next frame for either the current keypoint or the ball carrier
            if i + 1 >= len(neck_points_dict):
              opposite_direction_players_overlapping2.append(['frame '+str(i), point])
              break
            if neck_keypoints3_dict[point][i+1][0] == None:

              opposite_direction_players_overlapping2.append(['frame '+str(i), point])
            if neck_keypoints_ball_carrier[i+1][0] == None:

              opposite_direction_players_overlapping2.append(['frame '+str(i), point])

  return opposite_direction_players_overlapping2


#Function to run all previous functions of the system in one place

In [ ]:

# This function takes xball and yball coordinates as input
def systemRun(xball, yball):

  # Get the neck and hip keypoints of the ball carrier, as well as the list of ball carrier keypoints
  print(xball,yball)
  neck_keypoints_ball_carrier = getALLNeckAndHipKeypointsBallCarrier(path, xball, yball)[0]
  hip_keypoints_ball_carrier = getALLNeckAndHipKeypointsBallCarrier(path, xball, yball)[1]
  ball_carrier_index_list_global = getALLNeckAndHipKeypointsBallCarrier(path, xball, yball)[2]

  # Determine the overall direction of the ball carrier's movement
  direction_overall = get_speed(neck_keypoints_ball_carrier, hip_keypoints_ball_carrier)

  # Get the most frequent direction of the ball carrier's movement
  print(neck_keypoints_ball_carrier, hip_keypoints_ball_carrier)
  def most_frequent(List):
      return max(set(List), key = List.count)
  ball_carrier_direction_overall = most_frequent(direction_overall)

  # Get the poses of all persons in the scene
  all_persons_poses_results = getAllPersonsPoses()

  # Get the distinct people from the results
  distinct_people = all_persons_poses_results[0]

  # Get the neck and hip keypoints of each person
  x_neck_keypoints_dict = all_persons_poses_results[1]
  y_neck_keypoints_dict = all_persons_poses_results[2]
  x_hip_keypoints_dict = all_persons_poses_results[3]
  y_hip_keypoints_dict = all_persons_poses_results[4]

  # Combine the neck and hip keypoints into one dictionary for each person
  combine_XY_results = combineXYdict(x_neck_keypoints_dict, y_neck_keypoints_dict, x_hip_keypoints_dict, y_hip_keypoints_dict)
  neck_points_dict = combine_XY_results[0]
  hip_points_dict = combine_XY_results[1]

  # Separate the neck and hip keypoints into separate dictionaries for each person
  separate_keypoints_results = separateKeypointsEachId(distinct_people, x_neck_keypoints_dict, y_neck_keypoints_dict, x_hip_keypoints_dict, y_hip_keypoints_dict)
  neck_keypoints3_dict = separate_keypoints_results[0]
  hip_keypoints3_dict = separate_keypoints_results[1]

  # Determine which players are overlapping with the ball carrier and in the opposite direction of movement
  opposite_direction_players_overlapping = calculateNewOverlap(neck_points_dict, neck_keypoints3_dict, ball_carrier_index_list_global, ball_carrier_direction_overall, neck_keypoints_ball_carrier, hip_keypoints3_dict)

  # Initialize count and frame_outputs variables
  count = 0
  frame_outputs = []

  # Iterate through the overlapping players and determine the kind of tackle they make
  for i in opposite_direction_players_overlapping:

    # Get the frame of overlap and the ID of the tackler
    frame_of_overlap = opposite_direction_players_overlapping[count][0]
    tackler_id = opposite_direction_players_overlapping[count][1]

    # Get the number of the frame
    number_of_frame = frame_of_overlap.split(" ", 1)[1]

    # Get the height of the tackler and the ball carrier
    tackler_height = neck_keypoints3_dict[tackler_id][int(number_of_frame)][1]
    ball_carrier_height = neck_keypoints_ball_carrier[int(number_of_frame)][1]

    if tackler_height == None:
      continue
    if ball_carrier_height == None:
      continue
    if tackler_height <= ball_carrier_height:
      print("high tackle!")
      kind_of_tackle = "high tackle"
    else:
      print("non dangerous tackle")
      kind_of_tackle = "non dangerous tackle"
    count+=1
    frame_outputs.append([number_of_frame, tackler_height, ball_carrier_height, kind_of_tackle, tackler_id, ball_carrier_index_list_global])

  return frame_outputs


#Run the system on every video selected to produce file containing frame of tackle, risk of tackle, tackler ID and ball-carrier ID


In [ ]:
import json
from google.colab.patches import cv2_imshow
import cv2
from PIL import Image, ImageDraw

# define the output JSON file path
output_file = "coordinates.json"

# Load the JSON file
with open(output_file, 'r') as f:
    data = json.load(f)

src_dir = "/content/output_with_ids"
path = "/content/output"
count = 0
# Iterate through all the keys in the dictionary
for key in data.keys():

  !rm -r /content/output
  !mkdir /content/output


  # Retrieve the value for the current key
  value = data[key]

  # Get the xball and yball values from the value list
  x_ball, y_ball = value
  print(key)
  # Do something with the xball and yball variables, for example:
  print(f"xball: {x_ball}, yball: {y_ball}")

  # Find the corresponding subfolder name
  subfolder_name = key.split("/")[-1]
  print(subfolder_name)

  # Construct the full path to the subfolder
  subfolder_path = os.path.join(src_dir, subfolder_name)

  # Check if the subfolder exists
  if not os.path.exists(subfolder_path):
      print(f"Subfolder {subfolder_path} does not exist")
      continue
  print("folder found")
  print(subfolder_path)

  !cp -r $subfolder_path/. $path
  print(path)

  system_output = systemRun(x_ball,y_ball)

  print(system_output)
  tackle_result_list = []
  tackle_frame_list = []
  for i in range(len(system_output)):
    tackle_result_list.append(system_output[i][3])

    tackle_frame_list.append([system_output[i][0],system_output[i][3], system_output[i][4], system_output[i][5]])

  # define the output JSON file path
  tackle_file2 = "tackle_results.json"
  tackle_file3 = "tackle_frame_results.json"

  new_data2 = {key: tackle_result_list}
  new_data3 = {key: tackle_frame_list}

  print("data2:",new_data2)
  print("data3",new_data3)
  if count == 0:
    data2 = new_data2
    tackle_file2 = "/content/tackle_results.json"

    # Open the same file for writing
    with open(tackle_file2, 'w') as f:
        # Write the updated data to the file
        json.dump(data2, f)

    data3 = new_data3
    tackle_file3 = "/content/tackle_frame_results.json"
    # Open the same file for writing
    with open(tackle_file3, 'w') as f:
        # Write the updated data to the file
        json.dump(data3, f)

  else:
    # Open the JSON file for reading
    with open(tackle_file2, 'r+') as f:
        # Load the existing data from the file
        data2 = json.load(f)

    # Add new data to the dictionary
    new_data2 = {key: tackle_result_list}
    data2.update(new_data2)

    # Open the same file for writing
    with open(tackle_file2, 'w') as f:
        # Write the updated data to the file
        json.dump(data2, f)

    # Open the JSON file for reading
    with open(tackle_file3, 'r+') as f:
        # Load the existing data from the file
        data3 = json.load(f)

    # Add new data to the dictionary
    new_data3 = {key: tackle_frame_list}
    data3.update(new_data3)

    # Open the same file for writing
    with open(tackle_file3, 'w') as f:
        # Write the updated data to the file
        json.dump(data3, f)

  count+=1


/content/T1.mp4
xball: 690, yball: 400
T1.mp4
folder found
/content/output_with_ids/T1.mp4
/content/output
690 400
[(701.841, 388.438), (699.944, 390.366), (713.61, 398.09), (729.355, 396.171), (711.649, 396.188), (697.999, 400.191), (709.663, 411.931), (None, None), (713.59, 417.806), (None, None), (None, None), (None, None), (None, None), (None, None), (729.402, 417.809)] [(654.847, 396.23), (658.779, 394.266), (666.602, 398.235), (688.144, 392.337), (782.185, 402.125), (694.028, 443.205), (0, 0), (None, None), (774.435, 411.851), (None, None), (None, None), (None, None), (None, None), (None, None), (723.397, 462.813)]
high tackle!
high tackle!
high tackle!
[['2', 357.086, 398.09, 'high tackle', 7, [14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]], ['6', 380.554, 411.931, 'high tackle', 7, [14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]], ['14', 400.104, 417.809, 'high tackle', 7, [14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]]]
data2: {'/conten

#Calculate number of correctly identified tackles

In [ ]:
import csv
# initialize match and no_match counters
match_count = 0
no_match_count = 0
high_tackle = 0
low_tackle = 0
false_positives = 0
false_negatives = 0


with open('Tackle.csv') as f:
    reader = csv.reader(f)
    # get the column headers from the first row
    headers = next(reader)

# load the JSON file
with open('tackle_results.json') as f:
    data = json.load(f)
    # create a mapping between the column headers and the JSON keys
    mapping2 = []

    for key,value in data.items():
      mapping2.append(value)


# iterate over the columns in the CSV file and the keys in the JSON file
count = 0
for header in headers:
  # get the key in the JSON file that corresponds to the column header

  # iterate over the values in the column in the CSV file
  with open('Tackle.csv') as f:
      reader = csv.DictReader(f)

      for row in reader:
        value = row[header]
        print(row)

        # check if the value matches any of the values for the key in the JSON file
        print(mapping2)
        if value in mapping2[count]:
            index = mapping2[count].index(value)
            match_count += 1
            if value == "high tackle":
              high_tackle+=1
            else:
              low_tackle+=1

        else:
            no_match_count += 1
            if value == "non dangerous tackle":
              false_positives+=1
            if value == "high tackle":
              false_negatives+=1

  count+=1

# print the results
print('Matches:', match_count)
print('No matches:', no_match_count)
print('High:', high_tackle)
print('Non-dangerous:', low_tackle)
print(false_positives)
print(false_negatives)

{'High Tackle or Non Dangerous? (T1.mp4)': 'high tackle', 'High Tackle or Non Dangerous? (T2.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous? (T3.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous? (T4.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous? (T5.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous? (T6.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous? (T7.mp4)': 'high tackle', 'High Tackle or Non Dangerous? (T8.mp4)': 'high tackle', 'High Tackle or Non Dangerous? (T9.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous? (T10.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous? (T11.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous? (T12.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous? (T13.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous? (T14.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous? (T15.mp4)': 'non dangerous tackle', 'High Tackle or Non Dangerous?

IndexError: ignored

#Save tackle frames with both tackle heights and information about the tackle to new files

In [ ]:
!rm -r /content/openpose_test
!mkdir /content/openpose_test

rm: cannot remove '/content/openpose_test': No such file or directory


In [ ]:
import json
import cv2
import os
import os, glob
import re

!mkdir /content/openpose_test/

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

# read in JSON file with frame numbers and other values
with open('tackle_frame_results.json', 'r') as f:
    data = json.load(f)

# iterate over values in each key of the dictionary
for key in data:
    # extract frame number and video file name

    for i in range(len(data[key])):

      frame_number = data[key][i][0]

      video_file = key

      # extract other values from the list of lists
      value_1 = data[key][i][0]
      value_2 = data[key][i][1]
      value_3 = data[key][i][2]
      value_4 = data[key][i][3][0]
      # ...and so on
      src_dir = "/content/output_with_ids/"
      path = "/content/output"
      # count = 0


      !rm -r /content/output
      !mkdir /content/output

      # Find the corresponding subfolder name
      subfolder_name = video_file.split("/")[-1]
      print(subfolder_name)

      # Construct the full path to the subfolder
      subfolder_path = os.path.join(src_dir, subfolder_name)

      # Check if the subfolder exists
      if not os.path.exists(subfolder_path):
          print(f"Subfolder {subfolder_path} does not exist")
          continue
      print("folder found")
      print(subfolder_path)

      !cp -r $subfolder_path/. $path
      # extract neck locations and person IDs from OpenPose JSON file
      # read in OpenPose JSON file with neck locations and person IDs
      # loop through OpenPose JSON files in directory
      count = -1
      #####GET ALL PERSONS POSES#####

      for filename in sorted(glob.glob(os.path.join('/content/output', '*.json')), key=numericalSort):
        count+=1
        if count == int(frame_number):
          with open(filename, 'r') as f:
              openpose_data = json.load(f)
          neck_locations = []
          person_ids = []


          for person in openpose_data['people']:
            neck_location = person['pose_keypoints_2d'][3:5]
            person_id = person['person_id']
            neck_locations.append(neck_location)
            person_ids.append(person_id)

          # load video file and extract frame
          cap = cv2.VideoCapture(video_file)
          cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_number))
          ret, frame = cap.read()

          # add values and neck locations to frame
          font = cv2.FONT_HERSHEY_SIMPLEX
          font_scale = 1
          font_color = (255, 255, 255)
          thickness = 5
          count = 0
          for i, neck_location in enumerate(neck_locations):
              # draw circle at neck location

              cv2.circle(frame, tuple(map(int, neck_location)), 5, (0, 0, 255), -1)

              # add person ID to frame
              text = f"{person_ids[i]}"

              text_size, _ = cv2.getTextSize(text, font, font_scale, 1)
              text_x = int(neck_location[0] - text_size[0] / 2)
              text_y = int(neck_location[1] + text_size[1]+10)
              cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
              if person_ids[i] == value_3:
                color2 = (0, 0, 255) # Red color
                thickness2 = 2

                cv2.line(frame, (0, int(neck_location[1])), (frame.shape[1], int(neck_location[1])), color2, thickness2)
              if person_ids[i] == value_4:
                color2 = (0, 255, 0) # Green color
                thickness2 = 2

                cv2.line(frame, (0, int(neck_location[1])), (frame.shape[1], int(neck_location[1])), color2, thickness2)

          # add other values to frame
          text = f"Frame: {value_1}"
          cv2.putText(frame, text, (10, 30), font, font_scale, font_color, thickness, cv2.LINE_AA)

          text = f"Tackle Type: {value_2}"
          cv2.putText(frame, text, (10, 60), font, font_scale, font_color, thickness, cv2.LINE_AA)

          # add other values to frame
          text = f"Tackler: {value_3}"
          cv2.putText(frame, text, (10, 90), font, font_scale, font_color, thickness, cv2.LINE_AA)

          text = f"Ball-Carrier: {value_4}"

          cv2.putText(frame, text, (10, 120), font, font_scale, font_color, thickness, cv2.LINE_AA)

          # save frame to disk
          output_dir = '/content/openpose_test/'+subfolder_name.split(".")[0]+"frame"+value_1+'.png'
          cv2.imwrite(output_dir, frame)

          # release video capture object
          cap.release()

          break


mkdir: cannot create directory ‘/content/openpose_test/’: File exists
T1.mp4
folder found
/content/output_with_ids/T1.mp4
T1.mp4
folder found
/content/output_with_ids/T1.mp4
T1.mp4
folder found
/content/output_with_ids/T1.mp4
T2.mp4
folder found
/content/output_with_ids/T2.mp4
T2.mp4
folder found
/content/output_with_ids/T2.mp4
T2.mp4
folder found
/content/output_with_ids/T2.mp4
T3.mp4
folder found
/content/output_with_ids/T3.mp4
T3.mp4
folder found
/content/output_with_ids/T3.mp4
T3.mp4
folder found
/content/output_with_ids/T3.mp4
T3.mp4
folder found
/content/output_with_ids/T3.mp4
T4.mp4
folder found
/content/output_with_ids/T4.mp4
T4.mp4
folder found
/content/output_with_ids/T4.mp4
T5.mp4
folder found
/content/output_with_ids/T5.mp4
T5.mp4
folder found
/content/output_with_ids/T5.mp4
T5.mp4
folder found
/content/output_with_ids/T5.mp4
T5.mp4
folder found
/content/output_with_ids/T5.mp4
T5.mp4
folder found
/content/output_with_ids/T5.mp4
T6.mp4
folder found
/content/output_with_ids/T

#Adding the tackle frames to each video

In [ ]:
import cv2
from PIL import Image
import os
import shutil
import numpy as np

!mkdir /content/final_output/

video_dir = '/content/'
photo_dir = '/content/openpose_test'

for photo_name in os.listdir(photo_dir):
    photo_path = os.path.join(photo_dir, photo_name)

    video_name = photo_name[:photo_name.find('frame')]
    frame_name =  photo_name[photo_name.find('frame'):]
    print(frame_name.split(".")[0])
    video_name = video_name + '.mp4'
    video_path = os.path.join(video_dir, video_name)
    print(video_name, photo_path)

    if os.path.exists(photo_path):
        # Open the photo using Pillow and resize it to match the video size
        photo = Image.open(photo_path)

        # Open the video using OpenCV and get its properties
        video = cv2.VideoCapture(video_path)
        fps = video.get(cv2.CAP_PROP_FPS)
        width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Define the codec and output video file
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        path_name = "_output"+frame_name.split(".")[0]+".mp4"
        output_path = os.path.join('/content/final_output/', video_name.replace('.mp4', path_name))
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        # Loop through each frame of the video and write it to the output file
        while True:
            ret, frame = video.read()
            if not ret:
                break
            out.write(frame)

        # Write the photo to the end of the video
        for i in range(int(fps * 5)):  # add the photo for 5 seconds
            out.write(cv2.cvtColor(np.array(photo), cv2.COLOR_RGB2BGR))

        # Release the video and output file
        video.release()
        out.release()
    else:
        # Copy the video as is
        output_path = os.path.join("/content/final_output/", video_name.replace('.mp4', '_output.mp4'))
        shutil.copyfile(video_path, output_path)

frame11
T4.mp4 /content/openpose_test/T4frame11.png
frame14
T1.mp4 /content/openpose_test/T1frame14.png
frame9
T4.mp4 /content/openpose_test/T4frame9.png
frame23
T9.mp4 /content/openpose_test/T9frame23.png
frame24
T8.mp4 /content/openpose_test/T8frame24.png
frame6
T1.mp4 /content/openpose_test/T1frame6.png
frame39
T2.mp4 /content/openpose_test/T2frame39.png
frame32
T2.mp4 /content/openpose_test/T2frame32.png
frame11
T9.mp4 /content/openpose_test/T9frame11.png
frame20
T8.mp4 /content/openpose_test/T8frame20.png
frame17
T9.mp4 /content/openpose_test/T9frame17.png
frame46
T3.mp4 /content/openpose_test/T3frame46.png
frame13
T6.mp4 /content/openpose_test/T6frame13.png
frame23
T2.mp4 /content/openpose_test/T2frame23.png
frame21
T3.mp4 /content/openpose_test/T3frame21.png
frame2
T1.mp4 /content/openpose_test/T1frame2.png
frame16
T5.mp4 /content/openpose_test/T5frame16.png
frame41
T3.mp4 /content/openpose_test/T3frame41.png
frame54
T3.mp4 /content/openpose_test/T3frame54.png
frame13
T5.mp4 /co

In [ ]:
!zip -r /content/final_output.zip /content/final_output

  adding: content/final_output/ (stored 0%)
  adding: content/final_output/T3_outputframe46.mp4 (deflated 5%)
  adding: content/final_output/T2_outputframe39.mp4 (deflated 8%)
  adding: content/final_output/T9_outputframe11.mp4 (deflated 5%)
  adding: content/final_output/T4_outputframe11.mp4 (deflated 7%)
  adding: content/final_output/T6_outputframe10.mp4 (deflated 8%)
  adding: content/final_output/T7_outputframe17.mp4 (deflated 6%)
  adding: content/final_output/T1_outputframe2.mp4 (deflated 6%)
  adding: content/final_output/T8_outputframe20.mp4 (deflated 5%)
  adding: content/final_output/T2_outputframe23.mp4 (deflated 7%)
  adding: content/final_output/T9_outputframe23.mp4 (deflated 6%)
  adding: content/final_output/T1_outputframe6.mp4 (deflated 6%)
  adding: content/final_output/T5_outputframe16.mp4 (deflated 6%)
  adding: content/final_output/T2_outputframe32.mp4 (deflated 8%)
  adding: content/final_output/T3_outputframe54.mp4 (deflated 6%)
  adding: content/final_output/T1_

In [ ]:
from google.colab import files
files.download("/content/final_output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>